# Ejercicio Bow con PySpark
```
Docente: CARLOS FERNANDO MONTOYA CUBAS
Estudiante: ALEXANDER TTITO SAYA
Codigo: 164249
```

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 198 kB 87.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=528c43138c90769e3daedbbde5a1b89d6b944e6af33b852594ac161ae3ef1393
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [6]:
#son la libreias que se an a utilizar en todo el documento
import os
import pyspark
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import pandas as pd
pd.set_option('display.max_colwidth', 150)
import re
import string
import math
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

## Inicializamos un entorno de pyspark


In [8]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


##Modulo para procesar texto y incializacion de un corpus

In [12]:
# Implementamos el modulo BackOfWords para remover espacios,estandarizar letras a minusculas
def bagofwords(Doc):
    x=Doc.strip().lower().split()
    return list(filter(lambda x:len(x)>2,x))

In [43]:
#DEfinimos un Corpus
Corpus=[(1,"If you are not willing to risk the usual, you will have to settle for the ordinary"),
        (2,"Don’t count the days, make the days count"),(3,"The best way to predict the future is to invent it")]
sc =SparkContext.getOrCreate()
data=Corpus
#creamos una coleccion paralelizada para distribuir estos datos en varios nodos
lines=sc.parallelize(data)
#se muestran las oraciones ya procesadas(sin dobles espacios,simbolos,numeros)
lines.collect()


[(1,
  'If you are not willing to risk the usual, you will have to settle for the ordinary'),
 (2, 'Don’t count the days, make the days count'),
 (3, 'The best way to predict the future is to invent it')]

### Implementacion de TF

In [44]:
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
map1.collect()

[((1, 'If'), 1),
 ((1, 'you'), 1),
 ((1, 'are'), 1),
 ((1, 'not'), 1),
 ((1, 'willing'), 1),
 ((1, 'to'), 1),
 ((1, 'risk'), 1),
 ((1, 'the'), 1),
 ((1, 'usual,'), 1),
 ((1, 'you'), 1),
 ((1, 'will'), 1),
 ((1, 'have'), 1),
 ((1, 'to'), 1),
 ((1, 'settle'), 1),
 ((1, 'for'), 1),
 ((1, 'the'), 1),
 ((1, 'ordinary'), 1),
 ((2, 'Don’t'), 1),
 ((2, 'count'), 1),
 ((2, 'the'), 1),
 ((2, 'days,'), 1),
 ((2, 'make'), 1),
 ((2, 'the'), 1),
 ((2, 'days'), 1),
 ((2, 'count'), 1),
 ((3, 'The'), 1),
 ((3, 'best'), 1),
 ((3, 'way'), 1),
 ((3, 'to'), 1),
 ((3, 'predict'), 1),
 ((3, 'the'), 1),
 ((3, 'future'), 1),
 ((3, 'is'), 1),
 ((3, 'to'), 1),
 ((3, 'invent'), 1),
 ((3, 'it'), 1)]

Con la funcion `reduceByKey` fusionamos los valores de cada clave(clave/valor) y asi reducir la cantidad de datos en el proceso

In [47]:
reduce=map1.reduceByKey(lambda x,y:x+y)
reduce.collect()

[((1, 'If'), 1),
 ((1, 'you'), 2),
 ((1, 'not'), 1),
 ((1, 'to'), 2),
 ((1, 'the'), 2),
 ((1, 'will'), 1),
 ((1, 'for'), 1),
 ((1, 'ordinary'), 1),
 ((2, 'count'), 2),
 ((2, 'days,'), 1),
 ((2, 'make'), 1),
 ((2, 'days'), 1),
 ((3, 'to'), 2),
 ((3, 'the'), 1),
 ((3, 'future'), 1),
 ((3, 'it'), 1),
 ((1, 'are'), 1),
 ((1, 'willing'), 1),
 ((1, 'risk'), 1),
 ((1, 'usual,'), 1),
 ((1, 'have'), 1),
 ((1, 'settle'), 1),
 ((2, 'Don’t'), 1),
 ((2, 'the'), 2),
 ((3, 'The'), 1),
 ((3, 'best'), 1),
 ((3, 'way'), 1),
 ((3, 'predict'), 1),
 ((3, 'is'), 1),
 ((3, 'invent'), 1)]

Con la funcion `reduceByKey` fusionamos los valores de cada clave(clave/valor) y asi reducir la cantidad de datos en el proceso

In [ ]:
reduce=map1.reduceByKey(lambda x,y:x+y)
reduce.collect()

[((0, 'stage'), 1),
 ((0, 'protest'), 1),
 ((0, 'store'), 1),
 ((0, 'nyc'), 1),
 ((0, 'anger'), 1),
 ((0, 'misplace'), 1),
 ((0, 'retweet'), 1),
 ((1, 'apple'), 1),
 ((1, 'intraday'), 1),
 ((1, 'comment'), 1),
 ((1, 'range'), 1),
 ((1, 'premium'), 1),
 ((1, 'user'), 1),
 ((1, 'trading'), 1),
 ((1, 'stock'), 1),
 ((1, 'mkt'), 1),
 ((2, 'appname'), 1),
 ((2, 'elevate'), 1),
 ((2, 'training'), 1),
 ((4, 'boss'), 1),
 ((5, 'factcheckthis'), 1),
 ((6, 'educate'), 1),
 ((7, 'hard'), 1),
 ((7, 'reach'), 1),
 ((7, 'buy'), 1),
 ((7, 'squad'), 1),
 ((7, 'strip'), 1),
 ((7, 'apple'), 1),
 ((8, 'delete'), 1),
 ((8, 'music'), 1),
 ((8, 'customer'), 1),
 ((9, 'apple'), 1),
 ((9, 'intraday'), 1),
 ((9, 'comment'), 1),
 ((9, 'range'), 1),
 ((9, 'premium'), 1),
 ((9, 'user'), 1),
 ((9, 'trading'), 1),
 ((9, 'stock'), 1),
 ((9, 'mkt'), 1),
 ((10, 'battery'), 1),
 ((11, 'apple'), 1),
 ((11, 'great'), 1),
 ((12, 'love'), 1),
 ((12, 'reboot'), 1),
 ((13, 'positioning'), 1),
 ((13, 'fashion'), 1),
 ((13, 'g

utilizamos la funcion `map`  para idf

In [48]:
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

[('If', (1, 1)),
 ('you', (1, 2)),
 ('not', (1, 1)),
 ('to', (1, 2)),
 ('the', (1, 2)),
 ('will', (1, 1)),
 ('for', (1, 1)),
 ('ordinary', (1, 1)),
 ('count', (2, 2)),
 ('days,', (2, 1)),
 ('make', (2, 1)),
 ('days', (2, 1)),
 ('to', (3, 2)),
 ('the', (3, 1)),
 ('future', (3, 1)),
 ('it', (3, 1)),
 ('are', (1, 1)),
 ('willing', (1, 1)),
 ('risk', (1, 1)),
 ('usual,', (1, 1)),
 ('have', (1, 1)),
 ('settle', (1, 1)),
 ('Don’t', (2, 1)),
 ('the', (2, 2)),
 ('The', (3, 1)),
 ('best', (3, 1)),
 ('way', (3, 1)),
 ('predict', (3, 1)),
 ('is', (3, 1)),
 ('invent', (3, 1))]

In [50]:
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()
rdd=tf.join(idf)
rdd.collect()
#con map  ordenamos  los valores obetnidos (clave /valor)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()

spark = SparkSession(sc)
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
hasattr(rdd, "toDF")
# Si desea un DataFrame como salida, entonces necesita convertir el Dataset a DataFrame usando la  toDF() función
rdd.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).collect()

[Row(DocumentId=1, Token='are', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='willing', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='risk', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='usual,', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='have', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='you', TF=2, IDF=0.47712125471966244, TF-IDF=0.9542425094393249),
 Row(DocumentId=1, Token='to', TF=2, IDF=0.17609125905568124, TF-IDF=0.3521825181113625),
 Row(DocumentId=1, Token='the', TF=2, IDF=0.0, TF-IDF=0.0),
 Row(DocumentId=1, Token='settle', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='If', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(DocumentId=1, Token='not', TF=1, IDF=0.47712125471966244, TF-IDF=0.47712125471966244),
 Row(Documen